# This notebook is dedicated to the Exploratory Data Analysis of Research Project

In [1]:
# loading required packages
import pandas as pd
import numpy as np


In [4]:
# supportive functions
def dataset_dimension(name: str, dataset: pd.DataFrame):
    print(f"{name} dataset dimension: {dataset.shape}")

In [13]:
# load data
data_path = "data/"
train_identity = pd.read_csv(data_path + "train_identity.csv")
train_transaction = pd.read_csv(data_path + "train_transaction.csv")
test_identity = pd.read_csv(data_path + "test_identity.csv")
test_transaction = pd.read_csv(data_path + "test_transaction.csv")

In [14]:
# view each dataset dimension
dataset_dimension("Identity (train)", train_identity)
dataset_dimension("Transaction (train)", train_transaction)
dataset_dimension("Identity (test)", test_identity)
dataset_dimension("Transaction (test)", test_transaction)

Identity (train) dataset dimension: (144233, 41)
Transaction (train) dataset dimension: (590540, 394)
Identity (test) dataset dimension: (141907, 41)
Transaction (test) dataset dimension: (506691, 393)


In [10]:
train_transaction.head()


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# check for null values
